In [ ]:
import pandas as pd
import pickle

import ipywidgets as widgets
from ipywidgets import interact

from src.data.data_load import (
    load_tables, 
    load_online_instance, 
    load_distances, 
    upload_ONLINE_static_solution
)
from src.data.solution_load import load_solution_dfs
from src.utils.filtering import flexible_filter
from src.utils.plotting import plot_metrics_comparison_dynamic
from src.data.metrics import collect_results_to_df, compute_metrics_with_moves, get_day_plotting_df
from src.config.experimentation_config import *
from src.config.config import *

data_path = '../data'

instance = 'instRD3'        # Options: ['instAS1', 'instRS1']

distance_type = 'osrm'              # Options: ['osrm', 'manhattan']
dist_method = 'haversine'      # Options: ['precalced', 'haversine']

optimization_obj = 'driver_distance'

directorio_df, labors_raw_df, cities_df, duraciones_df, valid_cities = load_tables(data_path, generate_labors=False)
labors_real_df, labors_static_df, labors_dynamic_df = load_online_instance(data_path, instance, labors_raw_df)
dist_dict = load_distances(data_path, distance_type, instance, dist_method)

fechas = fechas_map(instance)

metricas = ['service_count', 'vt_count', 'num_drivers', 'driver_extra_time', 'driver_move_distance']

# Cargar resultados

In [13]:
algo_selection = {
    'hist': True,
    'offline': True,
    'INSERT': True,
    'INSERT_BUFFER': True,
    'REACT': True,
    'REACT_BUFFER': True
}

In [14]:
algorithms = []

if algo_selection['hist']:
    labors_hist_df, moves_hist_df, postponed_labors_hist = load_solution_dfs(
        data_path, 
        instance, 
        dist_method,
        algorithm='hist',
        include_all_city=True
    )
    algorithms.append({
        "name": "Histórico",
        "labors_df": labors_hist_df,
        "moves_df": moves_hist_df,
        "type": "historic",
        "color": "#C0392B",  # 🔴 Deep crimson red – “you were here”
        "visible": True,
    })

if algo_selection['offline']:    
    labors_algo_baseline_df, moves_algo_baseline_df, postponed_labors_OFFLINE = load_solution_dfs(
        data_path, 
        instance, 
        dist_method,
        algorithm='OFFLINE',
        include_all_city=True
    )
    algorithms.append({
        "name": "Offline",
        "labors_df": labors_algo_baseline_df,
        "moves_df": moves_algo_baseline_df,
        "type": "algorithm",
        "color": "#2980B9",  # 🔵 Royal blue – “best possible solution”
        "visible": True,
    })

labors_algo_static_df, moves_algo_static_df, postponed_labors_ONLINE_static = load_solution_dfs(
    data_path,
    instance,
    dist_method,
    algorithm='ONLINE_static',
    include_all_city=True
)

if algo_selection['INSERT']:
    labors_algo_INSERT_df, moves_algo_INSERT_df, postponed_labors_INSERT = load_solution_dfs(
        data_path,
        instance,
        dist_method,
        algorithm='INSERT',
        include_all_city=True
    )
    algorithms.append({
        "name": "INSERT",
        "labors_df": labors_algo_INSERT_df,
        "moves_df": moves_algo_INSERT_df,
        "type": "algorithm",
        "color": "#27AE60",  # 🟢 Vibrant green – “first smart improvement”
        "visible": True,
    })

if algo_selection['INSERT_BUFFER']:
    labors_algo_INSERT_BUFFER_df, moves_algo_INSERT_BUFFER_df, postponed_labors_INSERT_BUFFER = load_solution_dfs(
        data_path,
        instance,
        dist_method,
        algorithm='INSERT_BUFFER',
        include_all_city=True
    )
    algorithms.append({
        "name": "INSERT_BUFFER",
        "labors_df": labors_algo_INSERT_BUFFER_df,
        "moves_df": moves_algo_INSERT_BUFFER_df,
        "type": "algorithm",
        "color": "#16A085",
        "visible": True,
    })

if algo_selection['REACT']:
    labors_algo_REACT_df, moves_algo_REACT_df, postponed_labors_REACT_BUFFER_0 = load_solution_dfs(
        data_path, 
        instance, 
        dist_method,
        algorithm='REACT_BUFFER_0',
        include_all_city=True
    )
    algorithms.append({
        "name": "REACT",
        "labors_df": labors_algo_REACT_df,
        "moves_df": moves_algo_REACT_df,
        "type": "algorithm",
        "color": "#E67E22",  # 🟠 Bold orange – “fast adaptation”
        "visible": True,
    })

if algo_selection['REACT_BUFFER']:
    labors_algo_REACT_BUFFER_df, moves_algo_REACT_BUFFER_df, postponed_labors_REACT_BUFFER = load_solution_dfs(
        data_path, 
        instance, 
        dist_method,
        algorithm='REACT_BUFFER',
        include_all_city=True
    )
    algorithms.append({
        "name": "REACT_BUFFER",
        "labors_df": labors_algo_REACT_BUFFER_df,
        "moves_df": moves_algo_REACT_BUFFER_df,
        "type": "algorithm",
        "color": "#9B59B6",  # 🟣 Electric purple – “intelligent batching”
        "visible": True,
    })

# Resultados

In [ ]:


def plot_metrics_comparison_interactive_dynamic(
    algorithms, cities, metricas, dist_dict, fechas, metrics_with_gap=None
):
    """Fully dynamic interactive version for any number of algorithms."""

    city_dropdown = widgets.Dropdown(
        options=['ALL'] + sorted(list(set(cities))),
        value='ALL',
        description="Ciudad:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="300px")
    )

    # dynamically create visibility checkboxes
    algo_checks = {
        algo["name"]: widgets.Checkbox(value=algo.get("visible", True), description=algo["name"])
        for algo in algorithms
    }

    def _plot(city, **vis_flags):
        algo_copy = [{**algo, "visible": vis_flags[algo["name"]]} for algo in algorithms]
        plot_metrics_comparison_dynamic(
            algo_copy,
            city=city,
            metricas=metricas,
            dist_dict=dist_dict,
            fechas=fechas,
            metrics_with_gap=metrics_with_gap,
            save_dir=None
        )

    interact(
        _plot,
        city=city_dropdown,
        **algo_checks
    )

metrics_with_gap = [
    "total_distance",
    "driver_move_distance",
    "labor_extra_time",
    "driver_extra_time"
]

plot_metrics_comparison_interactive_dynamic(
    algorithms,
    cities=valid_cities,
    metricas=metricas,
    dist_dict=dist_dict,
    fechas=(fechas[0], fechas[-1]),
    metrics_with_gap=metrics_with_gap
)


interactive(children=(Dropdown(description='Ciudad:', layout=Layout(width='300px'), options=('ALL', '1', '1004…

# Validación de soluciones

In [5]:
from src.algorithms.solution_validator import validate_all_algorithms
validation_summary, moves_issues = validate_all_algorithms(
    algorithms[1:], 
    dist_method=dist_method,
    dist_dict=dist_dict
    )
display(validation_summary)



🚀 Validating Offline ...
🧑‍🔧 Checked 32 drivers, found 0 overlaps
✅ Validated 143 drivers, 1434 rows, 0 inconsistencies

🚀 Validating INSERT ...
🧑‍🔧 Checked 33 drivers, found 29 overlaps
✅ Validated 151 drivers, 1431 rows, 0 inconsistencies

🚀 Validating INSERT_BUFFER ...
🧑‍🔧 Checked 33 drivers, found 30 overlaps
✅ Validated 142 drivers, 1431 rows, 0 inconsistencies

🚀 Validating REACT ...
🧑‍🔧 Checked 33 drivers, found 1 overlaps
✅ Validated 140 drivers, 1401 rows, 0 inconsistencies

🚀 Validating REACT_BUFFER ...
🧑‍🔧 Checked 33 drivers, found 3 overlaps
✅ Validated 143 drivers, 1407 rows, 0 inconsistencies

📊 Validation Summary:


,algorithm,n_labors,n_moves,overlaps,move_inconsistencies,drivers_checked,rows_checked,pairs_checked,distance_calls,inconsistencies
0,Offline,527,1434,0,0,143,1434,1434,1434,0
1,INSERT,527,1431,29,0,151,1431,1431,1431,0
2,INSERT_BUFFER,527,1431,30,0,142,1431,1431,1431,0
3,REACT,487,1401,1,0,140,1401,1401,1401,0
4,REACT_BUFFER,490,1407,3,0,143,1407,1407,1407,0


In [7]:
labors_algo_INSERT_BUFFER_df

,service_id,labor_id,labor_type,labor_name,labor_category,labor_price,labor_created_at,labor_start_date,labor_end_date,alfred,...,labor_sequence,date,map_start_point,map_end_point,assigned_driver,actual_start,actual_end,dist_km,latest_arrival_time,n_drivers
0,268652,365656,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,55663.0,2025-07-14 08:30:06.116000-05:00,2025-07-21 06:36:00-05:00,2025-07-21 07:26:00-05:00,5455,...,0,2025-07-21,POINT (-75.56948779999999 6.203055700000001),POINT (-75.5697983 6.227549600000001),20111,2025-07-21 07:00:00-05:00,2025-07-21 07:49:05.143231-05:00,2.723814,NaT,NaN
16,268652,365656,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,55663.0,2025-07-14 08:30:06.116000-05:00,2025-07-21 06:36:00-05:00,2025-07-21 07:26:00-05:00,5455,...,0,2025-07-21,POINT (-75.56948779999999 6.203055700000001),POINT (-75.5697983 6.227549600000001),20111,2025-07-21 07:00:00-05:00,2025-07-21 07:49:05.143231-05:00,2.723814,NaT,NaN
36,268652,365656,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,55663.0,2025-07-14 08:30:06.116000-05:00,2025-07-21 06:36:00-05:00,2025-07-21 07:26:00-05:00,5455,...,0,2025-07-21,POINT (-75.56948779999999 6.203055700000001),POINT (-75.5697983 6.227549600000001),20111,2025-07-21 07:00:00-05:00,2025-07-21 07:49:05.143231-05:00,2.723814,NaT,NaN
52,268652,365656,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,55663.0,2025-07-14 08:30:06.116000-05:00,2025-07-21 06:36:00-05:00,2025-07-21 07:26:00-05:00,5455,...,0,2025-07-21,POINT (-75.56948779999999 6.203055700000001),POINT (-75.5697983 6.227549600000001),20111,2025-07-21 07:00:00-05:00,2025-07-21 07:49:05.143231-05:00,2.723814,NaT,NaN
1,269410,366473,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,49485.0,2025-07-15 12:53:50.757000-05:00,2025-07-21 07:44:00-05:00,2025-07-21 08:44:00-05:00,31857,...,0,2025-07-21,POINT (-75.616297 6.144),POINT (-75.564082 6.2082589),31857,2025-07-21 07:40:18.627282-05:00,2025-07-21 08:39:05.328823-05:00,9.185573,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,273675,371052,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,80663.0,2025-07-24 16:49:37.202000-05:00,2025-07-25 11:34:00-05:00,2025-07-25 13:25:00-05:00,6412,...,0,2025-07-25,POINT (-73.11345413684005 7.107574257887976),POINT (-73.1700487 7.066114),6412,2025-07-25 11:30:00-05:00,2025-07-25 12:26:38.605756-05:00,7.762286,NaT,NaN
4,273846,371238,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,55663.0,2025-07-25 09:41:02.027000-05:00,2025-07-25 13:46:00-05:00,2025-07-25 14:39:00-05:00,6412,...,0,2025-07-25,NaN,NaN,6412,2025-07-25 14:00:00-05:00,2025-07-25 14:51:15.726459-05:00,4.174738,2025-07-25 14:45:00-05:00,NaN
9,273846,371238,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,55663.0,2025-07-25 09:41:02.027000-05:00,2025-07-25 13:46:00-05:00,2025-07-25 14:39:00-05:00,6412,...,0,2025-07-25,NaN,NaN,6412,2025-07-25 14:00:00-05:00,2025-07-25 14:51:15.726459-05:00,4.174738,2025-07-25 14:45:00-05:00,NaN
10,274014,371416,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,77988.0,2025-07-25 13:23:06.810000-05:00,2025-07-25 15:32:00-05:00,2025-07-25 17:19:00-05:00,6412,...,0,2025-07-25,NaN,NaN,6412,2025-07-25 16:00:00-05:00,2025-07-25 16:57:47.459696-05:00,8.527330,2025-07-25 16:45:00-05:00,NaN


# Debugging

## Initial sanity checks

In [8]:
labors_1 = labors_algo_baseline_df[labors_algo_baseline_df['city'] != 'ALL'].dropna(subset=['actual_start']).copy()
labors_2 = labors_algo_INSERT_BUFFER_df[labors_algo_INSERT_BUFFER_df['city'] != 'ALL'].dropna(subset=['actual_start']).copy()

moves_1 = moves_algo_baseline_df[moves_algo_baseline_df['city'] != 'ALL'].dropna(subset=['actual_start']).copy()
moves_2 = moves_algo_INSERT_BUFFER_df[moves_algo_INSERT_BUFFER_df['city'] != 'ALL'].dropna(subset=['actual_start']).copy()

## Individual checks

In [11]:
city = '844'
fecha = '2026-01-05'

In [14]:
drop_columns = ['labor_type', 'labor_name', 'shop', 'alfred', 'labor_price', 
                'labor_created_at', 'labor_start_date', 'labor_end_date', 
                'client_type', 'paying_customer', 'state_service', 'labor_category',
                'start_address_id', 'end_address_id', 'date', 'address_id', 'address_point']
column_order = ['service_id', 'labor_id', 'created_at', 'schedule_date', 'actual_start', 'actual_end', 'map_start_point', 'map_end_point', 'city', 'assigned_driver', 'dist_km']

## NUMBER OF LABORS

In [17]:
filtered_labors_baseline = flexible_filter(labors_algo_baseline_df,
                city=city,
                schedule_date=fecha)

filtered_labors_static = flexible_filter(labors_algo_static_df,
                city=city,
                schedule_date=fecha)

filtered_labors_INSERT = flexible_filter(labors_algo_INSERT_df,
                city=city,
                schedule_date=fecha)

filtered_labors_INSERT_BUFFER = flexible_filter(labors_algo_INSERT_BUFFER_df,
                city=city,
                schedule_date=fecha)

filtered_labors_REACT = flexible_filter(labors_algo_REACT_df,
                city=city,
                schedule_date=fecha)

filtered_labors_REACT_BUFFER = flexible_filter(labors_algo_REACT_BUFFER_df,
                city=city,
                schedule_date=fecha)

filtered_moves_baseline = flexible_filter(moves_algo_baseline_df,
                city=city,
                schedule_date=fecha)

filtered_moves_INSERT = flexible_filter(moves_algo_INSERT_df,
                city=city,
                schedule_date=fecha)

filtered_moves_INSERT_BUFFER = flexible_filter(moves_algo_INSERT_BUFFER_df,
                city=city,
                schedule_date=fecha)

filtered_moves_REACT = flexible_filter(moves_algo_REACT_df,
                city=city,
                schedule_date=fecha)

filtered_moves_REACT_BUFFER = flexible_filter(moves_algo_REACT_BUFFER_df,
                city=city,
                schedule_date=fecha)

print(f'-------- Labors')
print(f'Baseline:      {len(filtered_labors_baseline)}')
print(f'INSERT:        {len(filtered_labors_INSERT)}')
print(f'INSERT BUFFER: {len(filtered_labors_INSERT_BUFFER)}')
print(f'REACT:         {len(filtered_labors_REACT)}')
print(f'REACT BUFFER:  {len(filtered_labors_REACT_BUFFER)}')

print(f'\n-------- Moves')
print(f'Baseline:      {len(filtered_moves_baseline)}')
print(f'INSERT:        {len(filtered_moves_INSERT)}')
print(f'INSERT BUFFER: {len(filtered_moves_INSERT_BUFFER)}')
print(f'REACT:         {len(filtered_moves_REACT)}')
print(f'REACT BUFFER:  {len(filtered_moves_REACT_BUFFER)}')

-------- Labors
Baseline:      4
INSERT:        4
INSERT BUFFER: 2
REACT:         4
REACT BUFFER:  4

-------- Moves
Baseline:      12
INSERT:        12
INSERT BUFFER: 6
REACT:         6
REACT BUFFER:  6


## MISSING LABORS/MOVES IN 844 05-01-25

### Labors

In [19]:
flexible_filter(
    labors_algo_baseline_df.drop(
        columns=drop_columns
    )[column_order],
    city=city, 
    schedule_date=fecha).sort_values(['schedule_date'])

,service_id,labor_id,created_at,schedule_date,actual_start,actual_end,map_start_point,map_end_point,city,dist_km
0,218033,311188,2026-01-03 08:22:06.741000-05:00,2026-01-05 07:00:00-05:00,2026-01-05 06:30:00-05:00,2026-01-05 07:17:09.137485-05:00,POINT (-73.11255469999999 7.117752199999999),POINT (-73.1192449 7.1288176),844,1.434861
1,218109,311287,2026-01-03 12:01:47.748000-05:00,2026-01-05 09:30:00-05:00,2026-01-05 09:00:00-05:00,2026-01-05 09:59:36.286581-05:00,POINT (-73.0782802 7.0281457),POINT (-73.11350980000002 7.108425699999999),844,9.736518
2,218905,312140,2026-01-05 11:06:20.985000-05:00,2026-01-05 16:00:00-05:00,2026-01-05 15:30:00-05:00,2026-01-05 16:22:59.420799-05:00,POINT (-73.0975205 7.0649086),POINT (-73.1149525 7.109582199999998),844,5.326898
3,218944,312188,2026-01-05 13:44:35.523000-05:00,2026-01-05 18:00:00-05:00,2026-01-05 17:30:00-05:00,2026-01-05 18:29:36.286581-05:00,POINT (-73.11350980000002 7.108425699999999),POINT (-73.0782802 7.0281457),844,9.736518


In [18]:
flexible_filter(
    labors_algo_INSERT_BUFFER_df.drop(
        columns=drop_columns
    )[column_order], 
    city=city, 
    schedule_date=fecha).sort_values(['schedule_date'])

,service_id,labor_id,created_at,schedule_date,actual_start,actual_end,map_start_point,map_end_point,city,dist_km
0,218033,311188,2026-01-03 08:22:06.741000-05:00,2026-01-05 07:00:00-05:00,2026-01-05 06:30:00-05:00,2026-01-05 07:17:09.137485-05:00,POINT (-73.11255469999999 7.117752199999999),POINT (-73.1192449 7.1288176),844,1.434861
1,218109,311287,2026-01-03 12:01:47.748000-05:00,2026-01-05 09:30:00-05:00,2026-01-05 09:00:00-05:00,2026-01-05 09:59:36.286581-05:00,POINT (-73.0782802 7.0281457),POINT (-73.11350980000002 7.108425699999999),844,9.736518


In [20]:
flexible_filter(
    labors_algo_REACT_df.drop(
        columns=drop_columns
    )[column_order], 
    city=city, 
    schedule_date=fecha).sort_values(['schedule_date'])

,service_id,labor_id,created_at,schedule_date,actual_start,actual_end,map_start_point,map_end_point,city,dist_km
0,218033,311188,2026-01-03 08:22:06.741000-05:00,2026-01-05 07:00:00-05:00,2026-01-05 06:30:00-05:00,2026-01-05 07:17:09.137485-05:00,POINT (-73.11255469999999 7.117752199999999),POINT (-73.1192449 7.1288176),844,1.434861
1,218109,311287,2026-01-03 12:01:47.748000-05:00,2026-01-05 09:30:00-05:00,2026-01-05 09:00:00-05:00,2026-01-05 09:59:36.286581-05:00,POINT (-73.0782802 7.0281457),POINT (-73.11350980000002 7.108425699999999),844,9.736518
2,218905,312140,2026-01-05 11:06:20.985000-05:00,2026-01-05 16:00:00-05:00,2026-01-05 15:30:00-05:00,2026-01-05 16:22:59.420799-05:00,POINT (-73.0975205 7.0649086),POINT (-73.1149525 7.109582199999998),844,5.326898
3,218944,312188,2026-01-05 13:44:35.523000-05:00,2026-01-05 18:00:00-05:00,2026-01-05 17:30:00-05:00,2026-01-05 18:29:36.286581-05:00,POINT (-73.11350980000002 7.108425699999999),POINT (-73.0782802 7.0281457),844,9.736518


### Moves

In [24]:
flexible_filter(
    moves_algo_baseline_df, 
    city=city, 
    schedule_date=fecha).sort_values(['schedule_date'])


,service_id,labor_id,labor_context_id,labor_name,labor_category,assigned_driver,schedule_date,actual_start,actual_end,start_point,end_point,distance_km,duration_min,city,date
0,218033,311188,311188_free,FREE_TIME,FREE_TIME,6412,2026-01-05 07:00:00-05:00,2026-01-05 06:23:45.091679-05:00,2026-01-05 06:23:45.091679-05:00,POINT (-73.1037471 7.0910492),POINT (-73.1037471 7.0910492),0.000000,0.0,844,2026-01-05
1,218033,311188,311188_move,DRIVER_MOVE,DRIVER_MOVE,6412,2026-01-05 07:00:00-05:00,2026-01-05 06:23:45.091679-05:00,2026-01-05 06:30:00-05:00,POINT (-73.1037471 7.0910492),POINT (-73.11255469999999 7.117752199999999),3.124236,6.2,844,2026-01-05
2,218033,311188,311188_labor,Alfred Initial Transport,VEHICLE_TRANSPORTATION,6412,2026-01-05 07:00:00-05:00,2026-01-05 06:30:00-05:00,2026-01-05 07:17:09.137485-05:00,POINT (-73.11255469999999 7.117752199999999),POINT (-73.1192449 7.1288176),NaN,47.2,844,2026-01-05
3,218109,311287,311287_free,FREE_TIME,FREE_TIME,6412,2026-01-05 09:30:00-05:00,2026-01-05 07:17:09.137485-05:00,2026-01-05 08:35:51.307394-05:00,POINT (-73.1192449 7.1288176),POINT (-73.1192449 7.1288176),0.000000,78.7,844,2026-01-05
4,218109,311287,311287_move,DRIVER_MOVE,DRIVER_MOVE,6412,2026-01-05 09:30:00-05:00,2026-01-05 08:35:51.307394-05:00,2026-01-05 09:00:00-05:00,POINT (-73.1192449 7.1288176),POINT (-73.0782802 7.0281457),12.072438,24.1,844,2026-01-05
5,218109,311287,311287_labor,Alfred Initial Transport,VEHICLE_TRANSPORTATION,6412,2026-01-05 09:30:00-05:00,2026-01-05 09:00:00-05:00,2026-01-05 09:59:36.286581-05:00,POINT (-73.0782802 7.0281457),POINT (-73.11350980000002 7.108425699999999),NaN,59.6,844,2026-01-05
6,218905,312140,312140_free,FREE_TIME,FREE_TIME,6412,2026-01-05 16:00:00-05:00,2026-01-05 09:59:36.286581-05:00,2026-01-05 15:19:41.939577-05:00,POINT (-73.11350980000002 7.108425699999999),POINT (-73.11350980000002 7.108425699999999),0.000000,320.1,844,2026-01-05
7,218905,312140,312140_move,DRIVER_MOVE,DRIVER_MOVE,6412,2026-01-05 16:00:00-05:00,2026-01-05 15:19:41.939577-05:00,2026-01-05 15:30:00-05:00,POINT (-73.11350980000002 7.108425699999999),POINT (-73.0975205 7.0649086),5.150504,10.3,844,2026-01-05
8,218905,312140,312140_labor,Alfred Initial Transport,VEHICLE_TRANSPORTATION,6412,2026-01-05 16:00:00-05:00,2026-01-05 15:30:00-05:00,2026-01-05 16:22:59.420799-05:00,POINT (-73.0975205 7.0649086),POINT (-73.1149525 7.109582199999998),NaN,53.0,844,2026-01-05
9,218944,312188,312188_free,FREE_TIME,FREE_TIME,6412,2026-01-05 18:00:00-05:00,2026-01-05 16:22:59.420799-05:00,2026-01-05 17:29:35.443094-05:00,POINT (-73.1149525 7.109582199999998),POINT (-73.1149525 7.109582199999998),0.000000,66.6,844,2026-01-05


In [23]:
flexible_filter(
    moves_algo_REACT_df, 
    city=city, 
    schedule_date=fecha).sort_values(['schedule_date'])

,service_id,labor_id,labor_context_id,labor_name,labor_category,assigned_driver,schedule_date,actual_start,actual_end,start_point,end_point,distance_km,duration_min,city,date
0,218033,311188,311188_free,FREE_TIME,FREE_TIME,6412,2026-01-05 07:00:00-05:00,2026-01-05 06:23:45.091679-05:00,2026-01-05 06:23:45.091679-05:00,POINT (-73.1037471 7.0910492),POINT (-73.1037471 7.0910492),0.000000,0.0,844,2026-01-05
1,218033,311188,311188_move,DRIVER_MOVE,DRIVER_MOVE,6412,2026-01-05 07:00:00-05:00,2026-01-05 06:23:45.091679-05:00,2026-01-05 06:30:00-05:00,POINT (-73.1037471 7.0910492),POINT (-73.11255469999999 7.117752199999999),3.124236,6.2,844,2026-01-05
2,218033,311188,311188_labor,Alfred Initial Transport,VEHICLE_TRANSPORTATION,6412,2026-01-05 07:00:00-05:00,2026-01-05 06:30:00-05:00,2026-01-05 07:17:09.137485-05:00,POINT (-73.11255469999999 7.117752199999999),POINT (-73.1192449 7.1288176),NaN,47.2,844,2026-01-05
3,218109,311287,311287_free,FREE_TIME,FREE_TIME,6412,2026-01-05 09:30:00-05:00,2026-01-05 07:17:09.137485-05:00,2026-01-05 08:35:51.307394-05:00,POINT (-73.1192449 7.1288176),POINT (-73.1192449 7.1288176),0.000000,78.7,844,2026-01-05
4,218109,311287,311287_move,DRIVER_MOVE,DRIVER_MOVE,6412,2026-01-05 09:30:00-05:00,2026-01-05 08:35:51.307394-05:00,2026-01-05 09:00:00-05:00,POINT (-73.1192449 7.1288176),POINT (-73.0782802 7.0281457),12.072438,24.1,844,2026-01-05
5,218109,311287,311287_labor,Alfred Initial Transport,VEHICLE_TRANSPORTATION,6412,2026-01-05 09:30:00-05:00,2026-01-05 09:00:00-05:00,2026-01-05 09:59:36.286581-05:00,POINT (-73.0782802 7.0281457),POINT (-73.11350980000002 7.108425699999999),NaN,59.6,844,2026-01-05


In [31]:
df = moves_algo_baseline_df
df = df[df['city'] != 'ALL']
df

,service_id,labor_id,labor_context_id,labor_name,labor_category,assigned_driver,schedule_date,actual_start,actual_end,start_point,end_point,distance_km,duration_min,city,date
0,205621,297525,297525_free,FREE_TIME,FREE_TIME,10491,2026-01-05 07:30:00-05:00,2026-01-05 06:53:04.404846-05:00,2026-01-05 06:53:04.404846-05:00,POINT (-75.5948708 6.222133500000001),POINT (-75.5948708 6.222133500000001),0.000000,0.0,1,2026-01-05
1,205621,297525,297525_move,DRIVER_MOVE,DRIVER_MOVE,10491,2026-01-05 07:30:00-05:00,2026-01-05 06:53:04.404846-05:00,2026-01-05 07:00:00-05:00,POINT (-75.5948708 6.222133500000001),POINT (-75.5722612 6.2005726),3.463293,6.9,1,2026-01-05
2,205621,297525,297525_labor,Alfred Initial Transport,VEHICLE_TRANSPORTATION,10491,2026-01-05 07:30:00-05:00,2026-01-05 07:00:00-05:00,2026-01-05 07:49:31.984874-05:00,POINT (-75.5722612 6.2005726),POINT (-75.56991479999999 6.2276503),NaN,49.5,1,2026-01-05
3,206697,298694,298694_free,FREE_TIME,FREE_TIME,31857,2026-01-05 08:00:00-05:00,2026-01-05 07:23:59.172324-05:00,2026-01-05 07:23:59.172324-05:00,POINT (-75.58290559999999 6.155980599999999),POINT (-75.58290559999999 6.155980599999999),0.000000,0.0,1,2026-01-05
4,206697,298694,298694_move,DRIVER_MOVE,DRIVER_MOVE,31857,2026-01-05 08:00:00-05:00,2026-01-05 07:23:59.172324-05:00,2026-01-05 07:30:00-05:00,POINT (-75.58290559999999 6.155980599999999),POINT (-75.5673676 6.1781754),3.006897,6.0,1,2026-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,215474,308380,308380_move,DRIVER_MOVE,DRIVER_MOVE,6412,2026-01-11 16:30:00-05:00,2026-01-11 15:55:44.773032-05:00,2026-01-11 16:00:00-05:00,POINT (-73.1037471 7.0910492),POINT (-73.11345413684005 7.107574257887976),2.126891,4.3,844,2026-01-11
11,215474,308380,308380_labor,Alfred Initial Transport,VEHICLE_TRANSPORTATION,6412,2026-01-11 16:30:00-05:00,2026-01-11 16:00:00-05:00,2026-01-11 16:46:46.377629-05:00,POINT (-73.11345413684005 7.107574257887976),POINT (-73.1063319 7.1155142),NaN,46.8,844,2026-01-11
6,215495,308403,308403_free,FREE_TIME,FREE_TIME,6412,2026-01-11 13:00:00-05:00,2026-01-11 10:19:15.875810-05:00,2026-01-11 12:14:23.509259-05:00,POINT (-73.1134749 7.1084515),POINT (-73.1134749 7.1084515),0.000000,115.1,844,2026-01-11
7,215495,308403,308403_move,DRIVER_MOVE,DRIVER_MOVE,6412,2026-01-11 13:00:00-05:00,2026-01-11 12:14:23.509259-05:00,2026-01-11 12:30:00-05:00,POINT (-73.1134749 7.1084515),POINT (-73.181381 7.128081700000001),7.804090,15.6,844,2026-01-11


In [33]:
df = moves_algo_REACT_df
df = df[df['city'] != 'ALL']
df[df['service_id'] == '218944']

,service_id,labor_id,labor_context_id,labor_name,labor_category,assigned_driver,schedule_date,actual_start,actual_end,start_point,end_point,distance_km,duration_min,city,date
9,218944,312188,312188_free,FREE_TIME,FREE_TIME,6412,2026-01-05 18:00:00-05:00,2026-01-05 16:22:59.420799-05:00,2026-01-05 17:29:35.443094-05:00,POINT (-73.1149525 7.109582199999998),POINT (-73.1149525 7.109582199999998),0.000000,66.6,nan,NaN
10,218944,312188,312188_move,DRIVER_MOVE,DRIVER_MOVE,6412,2026-01-05 18:00:00-05:00,2026-01-05 17:29:35.443094-05:00,2026-01-05 17:30:00-05:00,POINT (-73.1149525 7.109582199999998),POINT (-73.11350980000002 7.108425699999999),0.204641,0.4,nan,NaN
11,218944,312188,312188_labor,Alfred Initial Transport,VEHICLE_TRANSPORTATION,6412,2026-01-05 18:00:00-05:00,2026-01-05 17:30:00-05:00,2026-01-05 18:29:36.286581-05:00,POINT (-73.11350980000002 7.108425699999999),POINT (-73.0782802 7.0281457),NaN,59.6,nan,NaN
